In [8]:
import chess
from stockfish import Stockfish
import re
import requests
from ollama import chat
from ollama import ChatResponse
import json
import pandas as pd


In [9]:
stockfish = Stockfish()

In [10]:
csv_file = "/Users/nafis-mac/GitHub Repos/Chess-Research-Project/puzzles_code/filtered_fen.csv"

# Read the first 100 rows of the CSV file
df = pd.read_csv(csv_file, nrows=100)

# Convert the DataFrame rows to a list of lists
fen_notations = df["FEN"].tolist()

print(len(fen_notations))

100


In [11]:
def valid_move_to_string(board):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
]
    
    moves = str(legal_moves[0])  # Start with the first move

    for move in legal_moves[1:]:  # Skip the first move and iterate over the rest
        moves = moves + " " + move
    return moves

In [12]:
def get_phi_move(board:chess.Board, fen:str, temp=0.0):
    valid_moves = valid_move_to_string(board)
    
    response: ChatResponse = chat(model='llama3.3', messages=[
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn. This is the current state of the game. Use this to work out where the pieces are on the board:

FEN: {fen}

The possible set of legal moves are: 

Legal Moves: {valid_moves}

You have to choose one from the provided list. Do not choose a move that is not in the list.

Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.
{{
    "san": "The move in SAN format",
    "reason": "Why this is a good move"
}}"""}], options={"temperature": temp})
    
    try:
        parsed_json = json.loads(response.message.content)
        qwen_move = parsed_json["san"]
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")

    

    return qwen_move

In [13]:
results = []

In [14]:
puzzle_id = 1

for fen in fen_notations:
    print(f"Processing puzzle_id: {puzzle_id}\nPosition: {fen}")

    board_stockfish = chess.Board(fen=fen)
    board_phi = chess.Board(fen=fen)

    stockfish.set_fen_position(fen_position=fen)

    best_move = stockfish.get_best_move()
    best_move_san = board_stockfish.san(chess.Move.from_uci(best_move))

    board_stockfish.push(chess.Move.from_uci(best_move))
    stockfish.set_fen_position(fen_position=board_stockfish.fen())
    best_eval = stockfish.get_evaluation()  # evaluation in centipawns (or mate score)
        
    # print(f"Stockfish best move: {best_move} with evaluation: {best_eval}")
    if best_eval["type"] == "mate":
        break

    phi_move = get_phi_move(board_phi, fen, 0.6)

    try:
        board_phi.push_san(phi_move)
    except Exception as e:
        print(f"Exception captured: {type(e)}")
        continue

    stockfish.set_fen_position(fen_position=board_phi.fen())
    provided_eval = stockfish.get_evaluation()

    if provided_eval["type"] == "mate":
        # print("ITERATION SKIPPED")
        continue

    print(f"Stockfish best move: {best_move_san} with evaluation: {best_eval["value"]}")
    print(f"Llama move: {phi_move} with evaluation: {provided_eval["value"]}\n")

    results.append({
            # "Iteration": i+1,
            "Puzzle ID": puzzle_id,
            "Method": "Llama3.3",
            "Best Move": best_move_san,
            "Best Move Score": best_eval["value"],
            "Llama Move": {phi_move},
            "Llama Move Score": provided_eval["value"],
            # "Score Difference": score_difference
    })
    puzzle_id += 1

Processing puzzle_id: 1
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: Bxb5 with evaluation: 195
Llama move: Nf3 with evaluation: 77

Processing puzzle_id: 2
Position: rnbqkbnr/ppppp1pp/8/5p2/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: exf5 with evaluation: 156
Llama move: Nh3 with evaluation: 43

Processing puzzle_id: 3
Position: rnbqkbnr/1ppppppp/p7/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: d4 with evaluation: 75
Llama move: Nh3 with evaluation: -79

Processing puzzle_id: 4
Position: rnbqkbnr/p1pppppp/1p6/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: d4 with evaluation: 96
Llama move: Nh3 with evaluation: 7

Processing puzzle_id: 5
Position: rnbqkbnr/ppppp1pp/5p2/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: d4 with evaluation: 160
Llama move: Nh3 with evaluation: 108

Processing puzzle_id: 6
Position: rnbqkbnr/ppppppp1/7p/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move

In [15]:
df = pd.DataFrame(results)
df.to_csv("evaluation_results_llama3.3_noIter.csv", index=False)
print("Results saved to evaluation_results_llama3.3_noIter.csv")

Results saved to evaluation_results_llama3.3_noIter.csv
